In [ ]:
import pandas as pd

In [ ]:
def clean_dataframe(df, registry, start, end, verbose=False):

    df['Date'] = pd.to_datetime(df['Date'])

    registry['DateStop'] = pd.to_datetime(registry['DateStop'])

    data = df[(df.Date > pd.to_datetime(start)) & (df.Date < pd.to_datetime(end))]

    active = registry[(pd.isna(registry.DateStop)) | (registry.DateStop > pd.to_datetime(end))]

    IDs = list(set(active['IDStation']))

    data = data[data.IDStation.isin(IDs)].groupby('IDStation').mean()

    clean_df = pd.DataFrame(data=data.iloc[:,-1].values, index=data.index, columns=[data.columns[-1]])

    if verbose:
        print(clean_df.shape)
        print(clean_df.info())

In [ ]:
pollutants = ['NO2', 'NOx', 'PM2.5', 'PM10']

In [ ]:
registry = pd.read_csv('./data/registry.csv')

In [ ]:
for pollutant in pollutants:

    df = pd.read_csv('./data/raw/{}_sit_monthly.csv'.format(pollutant))

    clean_df = clean_dataframe(df, registry, 'March 2019', 'July 2019', verbose=True)